## Trabajo práctico 2
### Alumnos: Francisco Frusto Alvarado, Ezequiel Kaplan

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [51]:
#Inicializacion de pesos
W1 = np.random.random((5,6))
b1 = np.random.random((5,1))

W2 = np.random.random((1,5))
b2 = np.random.random((1,1))

theta = np.array([W1, b1, W2, b2])
print(theta)


[array([[0.48631474, 0.35667743, 0.06265077, 0.59212344, 0.97536497,
         0.50838568],
        [0.74135631, 0.24650511, 0.23600963, 0.90830446, 0.59812068,
         0.19029676],
        [0.74978488, 0.16930195, 0.12280759, 0.60550947, 0.38565293,
         0.77175354],
        [0.07067545, 0.07115243, 0.56132013, 0.25055902, 0.10209022,
         0.27460743],
        [0.86342984, 0.99726788, 0.36099777, 0.16570482, 0.10845809,
         0.13659808]])
 array([[0.09879222],
        [0.79837635],
        [0.44234104],
        [0.29710498],
        [0.48176168]])
 array([[0.21349149, 0.37504701, 0.66428418, 0.77375554, 0.46930586]])
 array([[0.66422576]])]


/var/folders/5d/4g206gss6cg3fktfbycqnb540000gn/T/ipykernel_27631/3391380950.py:8: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  theta = np.array([W1, b1, W2, b2])


Para implementar la funcion forward, vamos a hacer los cálculos de a pasos, sabiendo:

$\newline f_{\theta}(\mathrm{\mathbf{x}}) = W_{2} \ \sigma (W_{1} \mathrm{\mathbf{x}} + b_{1}) + b{2} $

En una primera instancia calculamos $ z_{1} = W_{1} \mathrm{\mathbf{x}} + b_{1}$ en la que hacemos el producto punto entre $\mathrm{\mathbf{x}}$ y $W_{1}$ y luego le sumamos el vector $b_{1}$. 

En un segundo paso, habiendo creado la funcion sigmoid que calcula: $\sigma(x) = \frac{1}{1 + e^{-x}}$ vamos a aplicarle esta funcion a los elementos de $z_{1}$

En un tercer paso, calculamos el resultado final en el que se hace el producto punto entre $W_{2}$ y el resultado anterior y ademas, se le suma el vector $b_{2}$

In [52]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def forward(theta, x):
    W1, b1, W2, b2 = theta

    z1 = np.dot(W1, x) + b1
    a1 = sigmoid(z1)
    z2 = np.dot(W2, a1) + b2

    return z2

La funcion obejtivo que buscamos minimizar, sabemos que está dada por: 
$\newline L = \frac{1}{2}(f_{\theta_{t}}(\mathrm{\mathbf{x}}_{i}) - y_{i})^2$

Para calcular el gradiente de forma numércia, vamos a usar la estrategia propuesta por la cátedra para calcular las derivadas parciales. 

In [62]:
#Calculo del gradiente numerico

def funcion_objetivo(theta, x, y):
    loss = 0.5 * (forward(theta, x) - y)**2
    return loss

def numerical_gradient(theta, x, y, eps):
    epsilon = eps  # Pequeña cantidad para el cálculo numérico del gradiente
    gradiente = np.zeros_like(theta)

    for i in range(len(theta)):
        theta_plus = theta.copy()
        theta_minus = theta.copy()

        # Aumentar y disminuir un poco el parámetro actual para calcular el gradiente
        theta_plus[i] += epsilon
        #print(f'el theta plus es: {theta_plus}')
        theta_minus[i] -= epsilon
        #print(f'el theta minus es: {theta_minus}')

        # Calcular las pérdidas para los parámetros aumentados y disminuidos
        loss_plus = funcion_objetivo(theta_plus, x, y)
        print(f'el loss plus es: {loss_plus}')
        loss_minus = funcion_objetivo(theta_minus, x, y)
        print(f'el loss minus es: {loss_minus}')

        # Calcular el gradiente parcial utilizando las derivadas parciales
        gradiente[i] = (loss_plus - loss_minus) / (2 * epsilon)

    print(f'el gradiente es: {gradiente}')
    return gradiente

In [63]:
#funcion fit y loop de entrenamiento
def fit(theta, x, y, learning_rate=0.001, epochs=1000):
    TOLERANCIA = 0.0001
    eps = 1e-5
    loss_accum = []

    for epoch in range(epochs):
        loss_epoch = 0.0  # Pérdida acumulada en el epoch actual

        for i in range(len(x)):
            # Obtener el vector x y la salida esperada y correspondientes
            x_i = x[i]
            y_i = y[i]

            # Calcular el gradiente numérico para el ejemplo actual
            gradient = numerical_gradient(theta, x_i, y_i, eps)

            # Transponer el gradiente para que tenga la forma (5, 1)
            gradient = gradient.T

            print(f'gradiente: {gradient}, theta: {theta}')

            # Actualizar los parámetros
            theta -= learning_rate * gradient

            # Calcular la pérdida para el ejemplo actual
            loss_i = funcion_objetivo(theta, x_i, y_i)
            loss_epoch += loss_i

        # Calcular la pérdida promedio para el epoch actual
        loss_avg = loss_epoch / len(x)
        loss_accum.append(loss_avg)

        if abs(theta - loss_avg) < TOLERANCIA:
            break

    return loss_accum, theta




In [60]:
def predict(x, theta):
        y = np.dot(x, theta)
        return y

In [64]:
# Entrenar la red neuronal
df_test = pd.read_excel("datos.xlsx", nrows=10)
x = df_test.iloc[:, 1:7].values.tolist()
y = df_test.iloc[:, 7].values.tolist()


loss_accum, theta_new = fit(theta, x, y, learning_rate=0.001, epochs=1000)

# Obtener predicciones en el conjunto de datos de prueba
y_pred = predict(x, theta_new)

# Calcular el error cuadrático medio en el conjunto de datos de prueba
mse = ((y_pred - y) ** 2).mean()

# Graficar la función objetivo a lo largo del entrenamiento
plt.plot(loss_accum)
plt.xlabel('Épocas')
plt.ylabel('Función Objetivo')
plt.title('Evolución de la función objetivo durante el entrenamiento')
plt.show()

# Graficar el error cuadrático medio en el conjunto de datos de prueba
plt.scatter(y, y_pred)
plt.xlabel('Valor Real')
plt.ylabel('Predicción')
plt.title('Predicciones vs. Valores Reales en el conjunto de datos de prueba')
plt.show()

print(f'Error cuadrático medio en el conjunto de datos de prueba: {mse}')


el loss plus es: [[603.429984 603.429984 603.429984 603.429984 603.429984]]
el loss minus es: [[603.429984 603.429984 603.429984 603.429984 603.429984]]
el loss plus es: [[603.429984 603.429984 603.429984 603.429984 603.429984]]
el loss minus es: [[603.429984 603.429984 603.429984 603.429984 603.429984]]
el loss plus es: [[603.429984 603.429984 603.429984 603.429984 603.429984]]
el loss minus es: [[603.429984 603.429984 603.429984 603.429984 603.429984]]
el loss plus es: [[603.429984 603.429984 603.429984 603.429984 603.429984]]
el loss minus es: [[603.429984 603.429984 603.429984 603.429984 603.429984]]
el gradiente es: [array([[0., 0., 0., 0., 0.]]) array([[0., 0., 0., 0., 0.]])
 array([[0., 0., 0., 0., 0.]]) array([[0., 0., 0., 0., 0.]])]
gradiente: [array([[0., 0., 0., 0., 0.]]) array([[0., 0., 0., 0., 0.]])
 array([[0., 0., 0., 0., 0.]]) array([[0., 0., 0., 0., 0.]])], theta: [array([[0.48631474, 0.35667743, 0.06265077, 0.59212344, 0.97536497,
         0.50838568],
        [0.7413

ValueError: operands could not be broadcast together with shapes (5,6) (1,5) 